In [1]:
import os

In [2]:
%pwd

'D:\\Developement\\Text-Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
# os.chdir("../")
# os.chdir("Developement/Text-Summarizer")

In [9]:
%pwd
os.chdir("src")

In [6]:
%pwd

'D:\\Developement\\Text-Summarizer'

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [24]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [26]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [27]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [28]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [32]:
%pwd
os.chdir("../")

In [33]:
%pwd

'D:\\Developement\\Text-Summarizer'

In [34]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-08-02 03:05:28,879: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-02 03:05:28,880: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-02 03:05:28,881: INFO: common: created directory at: artifacts]
[2024-08-02 03:05:28,882: INFO: common: created directory at: artifacts/data_ingestion]
[2024-08-02 03:05:31,895: INFO: 3078253932: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: E8A5:14681D:3F3525:4A230F:66ABFF9E
Accept-Ranges: bytes
Date: Thu, 01 Aug 2024 21:35:29 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4738-BOM
X-Cache: MI